In [5]:
import matplotlib
matplotlib.matplotlib_fname()
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

In [6]:
matplotlib.font_manager._rebuild()

In [3]:
def say_my_name(name):
    """
    Print the current widget value in short sentence
    """
    print(f'My name is {name}')
     
widgets.interact(say_my_name, name=["Jim", "Emma", "Bond"]);

interactive(children=(Dropdown(description='name', options=('Jim', 'Emma', 'Bond'), value='Jim'), Output()), _…

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from selenium.common.exceptions import NoSuchElementException   
import progressbar

options = Options()
options.headless = True
options.add_argument("--window-size=1920,10200") #this is important, to tell it how much of the webpage to import
driver = webdriver.Chrome(options=options, executable_path=r'/usr/local/bin/chromedriver')


In [2]:
def rose(link = 'https://leghe.fantacalcio.it/fantapalla-forever/area-gioco/rose?id=185855'):
    
    driver.get(link)
    rose = {}


    for j in range(1,9):
        test_1 = []
        name = (driver.find_element_by_xpath('/html/body/div[7]/main/div[3]/div[2]/div[1]/div[2]/div[2]/ul/li['+str(j)+']/div/div[1]/div[2]/h4').text).upper()
        players = driver.find_elements_by_xpath('/html/body/div[7]/main/div[3]/div[2]/div[1]/div[2]/div[2]/ul/li['+str(j)+']/table/tbody/tr[@class]/td[2]/a/b')
        for pl in players:
            test_1.append((pl.text).upper())

        rose[name] = test_1
        
        
    return pd.DataFrame.from_dict(data=rose, orient='index').T

In [3]:
def scarica_voti(giornata, stagione ='2020-21'):
    data = pd.read_excel('http://www.fantacalcio.it/Servizi/Excel.ashx?type=1&g='+str(giornata)+'&t=1601872625000&s='+stagione,skiprows = [0,1,2,3,4])
    data = data[data.Nome != 'Nome']
    data = data.dropna()
    data.index = list(range(len(data)))
    return data

In [4]:
def scarica_stats(stagione ='2020-21'):
    data = pd.read_excel('http://www.fantacalcio.it/Servizi/Excel.ashx?type=2&r=1&t=1604259046000&s='+stagione,skiprows = [0])
    data = data[data.Nome != 'Nome']
    data = data.dropna()
    data.index = list(range(len(data)))
    return data

In [5]:
dict_names={
    'AS 800A': 'enzo',
    'PDG 1908': 'pietro',
    'IGNORANZA EVERYWHERE': 'mario',
    'SOROS FC': 'musci8',
    'MAINZ NA GIOIA': 'franky',
    'PALLA PAZZA': 'nanni',
    'I DISEREDATI': 'emiliano',
    'XYZ': 'luca'
    }

def stats_by_team(stagione ='2020-21', dic = dict_names):
    Rose = rose()
    stats = scarica_stats(stagione)
    nomi = list(stats.Nome)
    stats.index = nomi
    stats_teams = []
    for team, df in Rose.items():
        for name in df:
            temp = list(stats.T[name]) + [team, dic[team]]
            col = list(stats.columns) + ['Nome Squadra', 'Allenatore']
            stats_teams.append(temp)
    return pd.DataFrame(data= stats_teams, columns = col)

In [6]:
stats_by_team()

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,Rc,R+,R-,Ass,Asf,Amm,Esp,Au,Nome Squadra,Allenatore
0,610,P,GOLLINI,Atalanta,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,XYZ,luca
1,159,P,SEPE,Parma,6,5.92,3.75,0,13,0,0,0,0,0,0,0,0,0,XYZ,luca
2,2211,P,SILVESTRI,Verona,5,6.40,6.00,0,2,0,0,0,0,0,0,0,0,0,XYZ,luca
3,1864,D,BRUNO ALVES,Parma,3,5.33,5.00,0,0,0,0,0,0,0,0,2,0,0,XYZ,luca
4,2759,D,CRISCITO,Genoa,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,XYZ,luca
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2819,A,CAPUTO,Sassuolo,5,7.00,10.40,4,0,0,1,1,0,2,0,0,0,0,IGNORANZA EVERYWHERE,mario
196,495,A,CORREA,Lazio,5,5.40,5.40,0,0,0,0,0,0,0,0,0,0,0,IGNORANZA EVERYWHERE,mario
197,2196,A,GALABINOV,Spezia,3,7.00,10.00,3,0,0,0,0,0,0,0,0,0,0,IGNORANZA EVERYWHERE,mario
198,568,A,QUAGLIARELLA,Sampdoria,5,6.60,8.40,3,0,0,2,1,1,0,0,0,0,0,IGNORANZA EVERYWHERE,mario
